In [1]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import itertools
from sklearn.metrics import *

In [2]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("y_train.csv")
sub = pd.read_csv("sample_submission.csv")

In [3]:
def fe(data):
    
    df = pd.DataFrame()
    data['totl_anglr_vel'] = (data['angular_velocity_X']**2 + data['angular_velocity_Y']**2 +
                             data['angular_velocity_Z']**2)** 0.5
    data['totl_linr_acc'] = (data['linear_acceleration_X']**2 + data['linear_acceleration_Y']**2 +
                             data['linear_acceleration_Z'])**0.5
    data['totl_xyz'] = (data['orientation_X']**2 + data['orientation_Y']**2 +
                             data['orientation_Z'])**0.5
   
    data['acc_vs_vel'] = data['totl_linr_acc'] / data['totl_anglr_vel']
    
    for col in data.columns:
        if col in ['row_id','series_id','measurement_number']:
            continue
        df[col + '_mean'] = data.groupby(['series_id'])[col].mean()
        df[col + '_median'] = data.groupby(['series_id'])[col].median()
        df[col + '_max'] = data.groupby(['series_id'])[col].max()
        df[col + '_min'] = data.groupby(['series_id'])[col].min()
        df[col + '_std'] = data.groupby(['series_id'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['series_id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_abs_max'] = data.groupby(['series_id'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['series_id'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
    return df

In [4]:
X_train = fe(X_train)
X_test = fe(X_test)
print(X_train.shape)

(3810, 154)


In [8]:
le = LabelEncoder()
y_train['surface'] = le.fit_transform(y_train['surface'])

In [5]:
X_train.fillna(0, inplace = True)
X_test.fillna(0, inplace = True)
X_train.replace(-np.inf, 0, inplace = True)
X_train.replace(np.inf, 0, inplace = True)
X_test.replace(-np.inf, 0, inplace = True)
X_test.replace(np.inf, 0, inplace = True)

In [6]:
def k_folds(X, y, X_test, k):
    folds = StratifiedKFold(n_splits = k, shuffle=True, random_state=2019)
    y_test = np.zeros((X_test.shape[0], 9))
    y_oof = np.zeros((X.shape[0]))
    score = 0
    for i, (train_idx, val_idx) in  enumerate(folds.split(X, y)):
        clf =  RandomForestClassifier(n_estimators = 500, n_jobs = -1)
        clf.fit(X_train.iloc[train_idx], y[train_idx])
        y_oof[val_idx] = clf.predict(X.iloc[val_idx])
        y_test += clf.predict_proba(X_test) / folds.n_splits
        score += clf.score(X.iloc[val_idx], y[val_idx])
        print('Fold: {} score: {}'.format(i,clf.score(X.iloc[val_idx], y[val_idx])))
    print('Avg Accuracy', score / folds.n_splits) 
        
    return y_oof, y_test 

In [9]:
y_oof, y_test = k_folds(X_train, y_train['surface'], X_test, k= 50)

/home/wwhlazio/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 21 members, which is too few. The minimum number of members in any class cannot be less than n_splits=50.
  % (min_groups, self.n_splits)), Warning)


Fold: 0 score: 0.9382716049382716
Fold: 1 score: 0.875
Fold: 2 score: 0.9259259259259259
Fold: 3 score: 0.9
Fold: 4 score: 0.8625
Fold: 5 score: 0.8625
Fold: 6 score: 0.925
Fold: 7 score: 0.8987341772151899
Fold: 8 score: 0.8589743589743589
Fold: 9 score: 0.8860759493670886
Fold: 10 score: 0.9743589743589743
Fold: 11 score: 0.8860759493670886
Fold: 12 score: 0.8481012658227848
Fold: 13 score: 0.8846153846153846
Fold: 14 score: 0.9342105263157895
Fold: 15 score: 0.9078947368421053
Fold: 16 score: 0.8571428571428571
Fold: 17 score: 0.8552631578947368
Fold: 18 score: 0.8947368421052632
Fold: 19 score: 0.8831168831168831
Fold: 20 score: 0.9090909090909091
Fold: 21 score: 0.935064935064935
Fold: 22 score: 0.8831168831168831
Fold: 23 score: 0.8289473684210527
Fold: 24 score: 0.9090909090909091
Fold: 25 score: 0.868421052631579
Fold: 26 score: 0.8961038961038961
Fold: 27 score: 0.9210526315789473
Fold: 28 score: 0.9342105263157895
Fold: 29 score: 0.92
Fold: 30 score: 0.868421052631579
Fold: 3

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_oof,y_train['surface'])

array([[156,   3,   0,   0,   4,   3,   2,   3,   0],
       [ 14, 702,  19,   1,  12,  16,   4,  22,  17],
       [  0,   7, 298,   0,   1,   3,   1,   2,   4],
       [  0,   0,   0,  10,   0,   0,   0,   0,   0],
       [  1,  13,   1,   0, 282,   4,   0,   3,   1],
       [  4,  15,   8,   0,   2, 677,   4,   8,  24],
       [  4,   9,   2,   2,   0,   8, 281,   3,   3],
       [  1,  10,  10,   0,   0,   3,   0, 464,  21],
       [  9,  20,  25,   8,   7,  18,   5,   9, 537]])

In [12]:
#y_test = np.argmax(y_test, axis=1)
sub['surface'] = le.inverse_transform(y_test)
sub.to_csv('submission_rf_new3.csv', index=False)